# Começamos com os imports

In [2]:
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import networkx as nx

# Carregamos os arquivos csv
Os arquivos csv com as publicações estão salvos no diretório data. As publicações nesses arquivos estão organizadas por linha com os campos separados por tabs. Como existem vários arquivos, é necessário agrupá-los em um único DataFrame usando o método _concat_.

In [29]:
df_raw = pd.concat(
      map(
        lambda file: pd.read_csv(file, 
                        sep='\t', 
                        header=0, 
                        encoding='utf-8-sig', 
                        index_col=False, 
                        dtype=str, 
                        na_values="", 
                        keep_default_na=False, 
                        na_filter=False), 
        glob.glob(os.path.join('data', "savedrecs-*.txt"))
        )
      )

print("Formato do DataFrame carregado", df_raw.shape)

Formato do DataFrame carregado (23862, 67)


# Limpeza dos dados
Os registros nos arquivos csv possuem algumas inconsistencias que precisam ser tratadas com algumas publicações com o ano de publicação vazio.
Aproveitamos para restringir nosso processamento aos anos desejados. Primeiro realizamos uma transformação no campos PY (Publication Year) para torná-lo numérico, definimos o intervalo de análise e em seguida aplicamos o filtro.
Salientamos que a função range usa o intervalo aberto a esquerda e fechado a direita.
Em seguida deletamos o DataFrame para evitar desperdício de recursos.

In [30]:
filter_years = list(range(1964, 2020))
    
df = df_raw[pd.to_numeric(df_raw.PY, downcast='integer').isin(filter_years)]
del df_raw
print("Formato do DataFrame após a limpeza", df.shape)

Formato do DataFrame após a limpeza (23432, 67)


# Seleção das características de interesse
Selecionamos apenas as caracteristicas de interesse para nossa análise, a lista de autores e o ano de cada publicação.

In [31]:
df_publications = df[['AF', 'PY', 'C1']]
del df
print("Formato do DataFrame de publicações", df_publications.shape)

Formato do DataFrame de publicações (23432, 3)


In [ ]:
G = nx.Graph(start=1964.0, end=2020.0)


def make_pairs(source):
    '''
    Returns an iterator with all combinations of 
    pair of elements in source
    '''
    result = []
    for p1 in range(len(source)):
        for p2 in range(p1+1,len(source)):
            result.append( (source[p1],source[p2]) )
    return result
        
for pub in df_publications.values:
    authors = list(map(str.strip, pub[0].split(';')))
    year = float(pub[1])
    affiliations = {}
    try:        
        if pub[2].find('[') < 0:
            for author_address in pub[2].split(';'):
                name = ''.join(author_address.split(',')[:2])
                address = ''.join(author_address.split(',')[2:]).strip()
                affiliations[name] = address
        else:
            authors_addr = pub[2].split('; [')
            for inner_author_addr in authors_addr:
                authrs,address = inner_author_addr.split('] ')
                for author in authrs.split(';'):
                    affiliations[author] = address.split(',')[0]
    except ValueError as err:
        print(f"Erro {err} ao encontrar filiação na publicação: {pub}")
    except AttributeError as err:
        print(f"Erro {err} ao encontrar filiação na publicação: {pub}")
    
    
    # Add nodes
    #G.add_nodes_from(authors)
    [G.add_node(a, affiliation = affiliations.get(a, ""), unb = "sim" if affiliations.get(a, "").startswith("Univ Brasilia")  else "nao") for a in authors]
    
    # Add edges
    for (source,target) in make_pairs(authors):
        if G.has_edge(source,target):
            years = set(G[source][target]['spells'])            
            years.add((year,year))
            G[source][target]['spells']=years           
        else:
            G.add_edge(source, target, spells= [(year,year)])

nx.write_gexf(G, 'wos-unb_xxx.gexf')

Erro not enough values to unpack (expected 2, got 1) ao encontrar filiação na publicação: ['Moura, Rodrigo Scaliante; Penna, Gerson Oliveira; Vaz Cardoso, Ludimila Paula; De Andrade Pontes, Maria Araci; Cruz, Rossilene; Goncalves, Heitor de Sa; Fernandes Penna, Maria Lucia; de Araujo Stefani, Mariane Martins; Buehrer-Sekula, Samira'
 '2015'
 'Univ Fed Goias, Inst Trop Pathol & Publ Hlth, Goiania, Go, Brazil; Univ Brasilia, Ctr Trop Med, Brasilia, DF, Brazil; Dona Libania Dermatol Ctr, Fortaleza, Ceara, Brazil; Alfredo da Matta Trop Dermatol & Venereol Fdn, Manaus, Amazonas, Brazil; Univ Fed Fluminense, Epidemiol & Biostat Dept, Rio De Janeiro, Brazil; [Moura, Rodrigo Scaliante; Vaz Cardoso, Ludimila Paula; de Araujo Stefani, Mariane Martins; Buehrer-Sekula, Samira] IPTSP, Setor Leste Univ, Dept Imunol, BR-74605050 Goiania, Go, Brazil; [Penna, Gerson Oliveira] Univ Brasilia, Nucleo Med Trop, Brasilia, DF, Brazil; [De Andrade Pontes, Maria Araci; Goncalves, Heitor de Sa] Ctr Referencia N

Erro not enough values to unpack (expected 2, got 1) ao encontrar filiação na publicação: ['Sawyer, Donald' '2008'
 'Univ Brasilia, Ctr Sustainable Dev, BR-70070 Brasilia, DF, Brazil; [Sawyer, Donald] ISPN, SCLN 202, BR-70832 Brasilia, DF, Brazil']
Erro not enough values to unpack (expected 2, got 1) ao encontrar filiação na publicação: ['Junqueira, Luiz Fernando, Jr.' '2008'
 'Univ Brasilia, Fac Med, Cardiovasc Lab, Clin Med Area, BR-70910900 Brasilia, DF, Brazil; [Junqueira, Luiz Fernando, Jr.] Univ Brasilia Hosp, Div Cardiol, BR-70910900 Brasilia, DF, Brazil']
Erro not enough values to unpack (expected 2, got 1) ao encontrar filiação na publicação: ['Patino, Jose Fernando; Goulart, Daniel Magalhaes' '2016'
 'Univ San Buenaventura Cali, Fac Psychol, Cali, Colombia; [Patino, Jose Fernando] Univ Brasilia, Fac Educ, BR-70910900 Brasilia, DF, Brazil; [Goulart, Daniel Magalhaes] Univ Ctr Brasilia, Brasilia, DF, Brazil; [Goulart, Daniel Magalhaes] Univ Brasilia, Fac Educ, CAPES, BR-7091090

In [19]:
import forceatlas2
import random
plt.rcParams["figure.figsize"] = (25,30)
pos = { i : (random.random(), random.random()) for i in G.nodes()}
l = forceatlas2.forceatlas2_networkx_layout(G, pos, niter=100)
nx.draw_networkx(G, l, node_size=50,
                 with_labels=False
                 #font_size=9
                )

#fig=plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')
 
# Optionally specify iteration count 
#nx.draw_networkx(G, l) 
plt.show()

KeyboardInterrupt: 

[('Palmeira, Ennio M.', {}), ('Beirigo, Elder A.', {}), ('Gardoni, Maria G.', {}), ('Nassar, Nagib M. A.', {}), ('Graciano-Ribeiro, Dalva', {}), ('Gomes, Paula F.', {}), ('Hashimoto, Danielle Y. C.', {}), ('Montagner, Maria Inez', {}), ('Montagner, Miguel Angelo', {}), ('Guimaraes, Marco Aurelio', {}), ('de Miranda, Bruno Eduardo C.', {}), ('Boiteux, Leonardo S.', {}), ('Reis, Ailton', {}), ('Dorea, Jose G.', {}), ('Shashikala, K.', {}), ('Sathyamoorthy, A.', {}), ('Raj, P.', {}), ('Dhar, S. K.', {}), ('Kundaliya, D.', {}), ('Malik, S. K.', {}), ('de Castro Silva, Isabella Monteiro', {}), ('Magalhaes, Isis Quezado', {}), ('Toscano, Raquel Alves', {}), ('Gandolfi, Lenora', {}), ('Pratesi, Riccardo', {}), ('Spaniol, E. P.', {}), ('De Andrade, V. C.', {}), ('Antunes, Adoniel W. S.', {}), ('Da Cunha, Wiliam Ferreira', {}), ('Silva, Geraldo Magela E.', {}), ('Martins, Joao B. L.', {}), ('Gargano, Ricardo', {}), ('Costa, A. A.', {}), ('Santos, P. M.', {}), ('Abib, F. C.', {}), ('Nassaralla, 

EdgeDataView([('Palmeira, Ennio M.', 'Beirigo, Elder A.', {'year': '2010'}), ('Palmeira, Ennio M.', 'Gardoni, Maria G.', {'year': '2010'}), ('Palmeira, Ennio M.', 'Antunes, Luiz G. S.', {'year': '2010'}), ('Beirigo, Elder A.', 'Gardoni, Maria G.', {'year': '2010'}), ('Nassar, Nagib M. A.', 'Graciano-Ribeiro, Dalva', {'year': '2010'}), ('Nassar, Nagib M. A.', 'Gomes, Paula F.', {'year': '2010'}), ('Nassar, Nagib M. A.', 'Hashimoto, Danielle Y. C.', {'year': '2010'}), ('Nassar, Nagib M. A.', 'Barbosa, Ionara Souza', {'year': '2010'}), ('Nassar, Nagib M. A.', 'Haridassan, M.', {'year': '2010'}), ('Nassar, Nagib M. A.', 'Ortiz, Rodomiro', {'year': '2010'}), ('Nassar, Nagib M. A.', 'Gomes, Pollyanna T. C.', {'year': '2010'}), ('Graciano-Ribeiro, Dalva', 'Gomes, Paula F.', {'year': '2010'}), ('Graciano-Ribeiro, Dalva', 'Hashimoto, Danielle Y. C.', {'year': '2010'}), ('Gomes, Paula F.', 'Hashimoto, Danielle Y. C.', {'year': '2010'}), ('Montagner, Maria Inez', 'Montagner, Miguel Angelo', {'yea